In [1]:
import opik

opik.configure(use_local=True, url='http://trainingvm:5173')

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# Create dataset
import opik
import pandas as pd
import requests
from io import BytesIO

client = opik.Opik()

# Create dataset
dataset = client.get_or_create_dataset(
    name="OpenAIModerationDataset", description="OpenAI Moderation Dataset"
)

# Insert items into dataset
url = "https://github.com/openai/moderation-api-release/raw/main/data/samples-1680.jsonl.gz"
response = requests.get(url)
df = pd.read_json(BytesIO(response.content), lines=True, compression="gzip")

df = df.sample(n=50, random_state=42)

dataset_records = []
for x in df.to_dict(orient="records"):
    moderation_fields = ["S", "H", "V", "HR", "SH", "S3", "H2", "V2"]
    moderated_fields = [field for field in moderation_fields if x[field] == 1.0]
    expected_output = "moderated" if moderated_fields else "not_moderated"

    dataset_records.append(
        {
            "output": x["prompt"],
            "expected_output": expected_output,
            "moderated_fields": moderated_fields,
        }
    )

dataset.insert(dataset_records)

OPIK: Created a "OpenAIModerationDataset" dataset at http://trainingvm:5173/api/v1/session/redirect/datasets/?dataset_id=0195c804-d3d8-763c-b1a9-5f8da455c8a0&path=aHR0cDovL3RyYWluaW5ndm06NTE3My9hcGkv.


In [4]:
from opik.evaluation.metrics import Moderation, Equals
from opik.evaluation import evaluate
from opik import Opik
from opik.evaluation.metrics.llm_judges.moderation.template import generate_query
from typing import Dict


# Define the evaluation task
def evaluation_task(x: Dict):
    metric = Moderation()
    try:
        metric_score = metric.score(output=x["output"])
        moderation_score = "moderated" if metric_score.value > 0.5 else "not_moderated"
        moderation_reason = metric_score.reason
    except Exception as e:
        print(e)
        moderation_score = None
        moderation_reason = str(e)

    return {
        "moderation_score": moderation_score,
        "moderation_reason": moderation_reason,
    }


# Get the dataset
client = Opik()
dataset = client.get_dataset(name="OpenAIModerationDataset")

# Define the scoring metric
moderation_metric = Equals(name="Correct moderation score")

# Add the prompt template as an experiment configuration
experiment_config = {
    "prompt_template": generate_query(output="{output}", few_shot_examples=[])
}

res = evaluate(
    dataset=dataset,
    task=evaluation_task,
    scoring_metrics=[moderation_metric],
    experiment_config=experiment_config,
    scoring_key_mapping={"reference": "expected_output", "output": "moderation_score"},
)

Evaluation:   0%|          | 0/50 [00:00<?, ?it/s]OPIK: Started logging traces to the "Default Project" project at http://trainingvm:5173/api/v1/session/redirect/projects/?trace_id=0195c805-5b49-76ad-862a-b982c9b3cf81&path=aHR0cDovL3RyYWluaW5ndm06NTE3My9hcGkv.
OPIK: Started logging traces to the "Default Project" project at http://trainingvm:5173/api/v1/session/redirect/projects/?trace_id=0195c805-5b3b-71fb-8f23-4491e54c3013&path=aHR0cDovL3RyYWluaW5ndm06NTE3My9hcGkv.
Evaluation: 100%|██████████| 50/50 [00:15<00:00,  3.15it/s]


╭─ OpenAIModerationDataset (50 samples) ─╮
│                                        │
│ Total time:        00:00:16            │
│ Number of samples: 50                  │
│                                        │
│ Correct moderation score: 0.8400 (avg) │
│                                        │
╰────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=276060;http://trainingvm:5173/api/v1/session/redirect/experiments/?experiment_id=0195c805-5a37-7fa2-bb86-aa38f555114c&dataset_id=0195c804-d3d8-763c-b1a9-5f8da455c8a0&path=aHR0cDovL3RyYWluaW5ndm06NTE3My9hcGkv\in your Opik dashboard]8;;\.